In [4]:
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [5]:
producer = KafkaProducer(bootstrap_servers=['192.168.1.100:9092'])  # boostrap_servers = 架設Kafka伺服器的主機IP(內網用Private IP，外網(外部主機)用Public IP)，Kafka Server服務默認端口為9092

In [6]:
# asynchronous by default
future = producer.send('NanmenBridge', b'raw_bytes') # 通常，進行網路傳輸時，資料格式皆為二進位格式(bytes)
print(future)

In [7]:
# block for 'synchronous' sends
try:
    record_metadata = future.get(timeout=10)
    print(record_metadata)
except KafkaError:
    # decide what to do if produce request failed...
    pass

RecordMetadata(topic='NanmenBridge', partition=0, topic_partition=TopicPartition(topic='NanmenBridge', partition=0), offset=8081, timestamp=1692343576435, log_start_offset=4, checksum=None, serialized_key_size=-1, serialized_value_size=9, serialized_header_size=-1)


In [9]:
# successful result returns assigned partition and offset
print(record_metadata.topic) # 訊息傳遞的Topic 名稱
print(record_metadata.partition) # 訊息傳送到Topic內哪一個Partition，如果當初建立Topic時只有一個Partition，則Partition 編號 = 0
print(record_metadata.offset) # 訊息傳到Topic內特定的Partition中的位置 = 資料id ， 最舊的訊息(資料) : offset(id) = 0，最新的訊息 : 則offset(id) 越大 

NanmenBridge
0
8081


In [10]:
# produce keyed messages to enable hashed partitioning
producer.send('my-NanmenBridge-topic', key=b'foo', value=b'bar')

## **批次(Batch)發布(Publish)訊息** 
Batch = 幾筆資料一綑 -> 一次將多筆資料綑成一綑，發送至Kafka Server 

like 深度學習的batch -> 幾筆資料一綑，並進行訓練，進行Batch Learning

Agument : 
1. linger_ms : 等待幾毫秒(ms)，當超過設定時間，則在時間內傳送的訊息一次送出去
2. batch_size : 設定一綑的筆數，如果傳送的訊息樹木超過一綑的數量，則傳送出去

優先順序 : batch_size > linger_ms ， 兩個都設置參數的情況下，如果在限定時間(linger_ms)內達到一綑的數目(batch_size)，則會送出，否則就會等到限定時間後才送出

In [12]:
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers=['192.168.1.100:9092'] , 
                         linger_ms = 1000,
                         batch_size = 1000)  # boostrap_servers = 架設Kafka伺服器的主機IP(內網用Private IP，外網(外部主機)用Public IP)，Kafka Server服務默認端口為9092

for _ in range(1000):
    future = producer.send('NanmenBridge', b'Hello Kafka!')

# 這些訊息都會存到緩衝區，直到訊息筆數超過限制條件(linger_ms、batch_size)，才會一次送出
producer.flush() # 確保緩衝區的資料全部傳送到Kafka伺服器

In [13]:
future.get()

RecordMetadata(topic='NanmenBridge', partition=0, topic_partition=TopicPartition(topic='NanmenBridge', partition=0), offset=10081, timestamp=1692347585647, log_start_offset=4, checksum=None, serialized_key_size=-1, serialized_value_size=12, serialized_header_size=-1)